In [18]:
import random
import pandas as pd 
from PIL import Image

import torch
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim 

from torchmetrics.classification import BinaryF1Score
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

In [19]:
class DeepfakeDataset(Dataset): 
    def __init__(self, csv_file, root_dir, transform, train=bool): 
        self.data = pd.read_csv(csv_file) 
        self.root = root_dir 
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(self.data) 

    def __getitem__(self, idx): 
        if self.train: 
            img_path = self.root + self.data.iloc[idx, 1]
            image = Image.open(img_path).convert("RGB") 
            image = self.transform(image) 
            label = self.data.iloc[idx, 2]

            return image, torch.tensor(label)
        else:
            img_path = self.root + self.data.iloc[idx, 0]
            image = Image.open(img_path).convert("RGB") 
            image = self.transform(image) 

            return image

In [20]:
img_width = (128) // (2*2)
img_height = (128) // (2*2)

class AIDetector(nn.Module):
    def __init__(self): 
        super().__init__()

        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(), 
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(16, 32, 3, padding=1),
            nn.BatchNorm2d(32), 
            nn.ReLU(), 
            nn.MaxPool2d(2, 2), 
        )

        self.fc = nn.Sequential(
            nn.Flatten(), 
            
            nn.Linear(32 * img_width * img_height, 512),
            nn.ReLU(),

            nn.Linear(512, 64),
            nn.ReLU(), 

            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x): 
        x = self.conv_block(x)
        x = self.fc(x) 

        return x 

In [21]:
def process_fold(model, train_loader, val_loader, optimizer, criterion, device, epochs): 
    train_f1_metric = val_f1_metric = BinaryF1Score().to(device)
    train_scores = val_scores = []

    for epoch in range(epochs):
        model.train()
        train_f1_metric.reset()
        
        for img, label in tqdm(train_loader, desc=f"Epoch: {epoch+1}"): 
            img, label = img.to(device), label.to(device)            
            optimizer.zero_grad()
            outputs = model(img).squeeze()  

            loss = criterion(outputs, label.float()) 
            loss.backward()
            optimizer.step() 

            preds = (outputs > 0.5).int()  
            train_f1_metric.update(preds, label.int()) 

        epoch_train_f1 = train_f1_metric.compute()  
        train_scores.append(epoch_train_f1.item()) 
        
        model.eval() 
        with torch.no_grad():
            val_f1_metric.reset() 
            for img, label in val_loader:
                img, label = img.to(device), label.to(device)
                
                outputs = model(img).squeeze()
                preds = (outputs > 0.5).int() 
                
                val_f1_metric.update(preds, label.int()) 
    
        epoch_val_f1 = val_f1_metric.compute()
        val_scores.append(epoch_val_f1.item())  

        print(f"Train F1 Score: {epoch_train_f1:.3f}, Val F1 Score: {epoch_val_f1:.3f}")

    avg_train_f1 = sum(train_scores) / len(train_scores)  
    avg_val_f1 = sum(val_scores) / len(val_scores) 
    
    return avg_train_f1, avg_val_f1

def train_with_kfold(model, dataset, criterion, optimizer, device, k_folds=3, epochs=5):
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    train_f1 = val_f1 = [] 
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)): 
        print(f"TRAINING FOLD {fold + 1}/{k_folds}\n")

        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
        
        fold_train_f1, fold_val_f1 = process_fold(
            model,  
            train_loader, 
            val_loader, 
            optimizer, 
            criterion, 
            device,
            epochs
        )
        
        print(f"\nFOLD {fold + 1} - TRAIN F1_SCORE: {fold_train_f1:.3f}, "
              f"VAL F1_SCORE: {fold_val_f1:.3f}\n")
        
        train_f1.append(fold_train_f1)
        val_f1.append(fold_val_f1)

    avg_train_f1 = sum(train_f1) / len(train_f1)
    avg_val_f1 = sum(val_f1) / len(val_f1)

    print(f"{'-'*25}\n")
    print(f"AVG TRAIN F1_SCORE: {avg_train_f1:.3f}, "
          f"AVG VAL F1_SCORE: {avg_val_f1:.3f}")

    return avg_train_f1, avg_val_f1

In [22]:
def eval(model, test_loader, criterion, device):
    model.eval() 
    all_preds = [] 

    with torch.no_grad(): 
        for img in test_loader:
            img = img.to(device)
            img.flatten(start_dim=1)

            outputs = model(img).squeeze()
            preds = (outputs > 0.5).int()
            all_preds.extend(preds.cpu().numpy())

    return all_preds

In [23]:
transform = transforms.Compose([
    transforms.Resize((128, 128)), 
    transforms.ToTensor()
])

train_data = DeepfakeDataset(
    csv_file="/kaggle/input/ai-vs-human-generated-dataset/train.csv", 
    root_dir="/kaggle/input/ai-vs-human-generated-dataset/",
    transform=transform,
    train=True
)

test_data = DeepfakeDataset(
    csv_file="/kaggle/input/ai-vs-human-generated-dataset/test.csv",
    root_dir="/kaggle/input/ai-vs-human-generated-dataset/", 
    transform=transform,
    train=False
)

test_loader = DataLoader(
    test_data, 
    batch_size=32, 
    shuffle=False
)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AIDetector().to(device)
epochs = 3

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()  

avg_train_f1, avg_test_f1 = train_with_kfold(
    model, 
    train_data,
    criterion,
    optimizer, 
    device,
    k_folds=3, 
    epochs=5
)

TRAINING FOLD 1/3



Epoch: 1: 100%|██████████| 1666/1666 [09:27<00:00,  2.93it/s]


Train F1 Score: 0.884, Val F1 Score: 0.863


Epoch: 2: 100%|██████████| 1666/1666 [05:59<00:00,  4.63it/s]


Train F1 Score: 0.917, Val F1 Score: 0.903


...

Train F1 Score: 0.991, Val F1 Score: 0.991


Epoch: 4: 100%|██████████| 1666/1666 [06:15<00:00,  4.43it/s]


Train F1 Score: 0.993, Val F1 Score: 0.990


Epoch: 5: 100%|██████████| 1666/1666 [05:40<00:00,  4.89it/s]


Train F1 Score: 0.994, Val F1 Score: 0.986

FOLD 3 - TRAIN F1_SCORE: 0.987, VAL F1_SCORE: 0.987

-------------------------

AVG TRAIN F1_SCORE: 0.962, AVG VAL F1_SCORE: 0.962


In [36]:
ids = pd.read_csv("/kaggle/input/ai-vs-human-generated-dataset/test.csv")["id"]
preds = eval(model, test_loader, criterion, device)

submission = pd.DataFrame({
    "id": ids,
    "label": preds
})

submission.to_csv("/kaggle/working/V1.csv", index=False)

In [32]:
model.train()
torch.save(model.state_dict(), "weights_V1.pth")

In [38]:
submission['label'].value_counts()

label
0    4967
1     573
Name: count, dtype: int64